# Steady State - Households

**Table of contents**<a id='toc0_'></a>    
- 1. [Setup](#toc1_)    
- 2. [Mortality and population](#toc2_)    
- 3. [Labor supply](#toc3_)    
- 4. [Consumption-saving](#toc4_)    
- 5. [Varying central inputs](#toc5_)    
- 6. [Test household blocks](#toc6_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
%load_ext autoreload
%autoreload 2

import time
import pickle
import numpy as np
from scipy import optimize
import ipywidgets as widgets

import matplotlib.pyplot as plt   
plt.style.use('seaborn-v0_8-whitegrid')
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

from BabyMAKROModel import BabyMAKROModelClass

C:\Users\nbk863\AppData\Local\Temp\ipykernel_6828\3304960369.py:11: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


In [2]:
from steady_state import household_search_ss, find_household_consumption_ss, find_Aq_ss     #import household steady state functions 
from blocks import search_and_match, household_consumption                                  #import household blocks functions

## 1. <a id='toc1_'></a>[Setup](#toc0_)

In [3]:
model = BabyMAKROModelClass(name='baseline')    #create an instance of the model

In [4]:
par = model.par     #set model parameters
ss = model.ss       #create namespace for steady state
sol = model.sol     #create namespace for solution

## 2. <a id='toc2_'></a>[Mortality and population](#toc0_)

In [ ]:
def Mortality_and_population_figure(zeta_pow):
    model.mortality(zeta_pow)
    model.demographic_structure(par.delta_L_a_fac)

    
    fig = plt.figure(figsize=(3*6,6/1.5))

    ax = fig.add_subplot(1,3,1)
    ax.plot(par.zeta_a*100)
    ax.set_ylabel('%')
    ax.set_title('Mortality rate, $\zeta_a$')

    ax = fig.add_subplot(1,3,2)
    ax.plot(par.N_a)
    RetirementAge = ax.vlines(par.work_life_span, 0, 1.05, colors='orange',linestyles='dashed', label='Retirement age')
    ax.legend(handles = [RetirementAge])
    ax.set_title('Population, $N_a$')

    return print(f'Average life expectancy: {par.N = :.1f}'), print(f'Population size of the workforce: {par.N_work = :.1f}')

In [ ]:
widgets.interact(Mortality_and_population_figure,
    zeta_pow=widgets.FloatText(description="Zeta_power", value=par.zeta_pow));

**Reset demographic structure:**

In [ ]:
model.mortality(par.zeta_pow)                       #reset the mortaility rates
model.demographic_structure(par.delta_L_a_fac)      #reset the demographic structure

## 3. <a id='toc3_'></a>[Labor supply](#toc0_)

**Required inputs:**

In [5]:
ss.m_s = par.m_v_ss                    #set steady state for the job-finding rate

In [6]:
household_search_ss(par,ss)            #find the household search behavior in steady state

In [ ]:
def household_search_figure(delta_L_a_fac):
    model.demographic_structure(delta_L_a_fac)
    household_search_ss(par,ss)   
    fig = plt.figure(figsize=(3*6,6/1.5))

    ax = fig.add_subplot(1,3,1)
    ax.plot(par.delta_L_a*100)
    ax.set_ylabel('%')
    ax.set_ylim([0, 105])
    ax.set_title('Separation rate, $\delta^L_a$');

    ax = fig.add_subplot(1,3,2)
    ax.plot(ss.S_a)
    ax.set_ylim([0, 1.05])
    ax.set_title('Searchers, $S_a$')

    ax = fig.add_subplot(1,3,3)
    ax.plot(ss.L_a)
    ax.set_ylim([0, 1.05])
    employed = ax.fill_between(range(0,par.work_life_span),ss.L_a[0:par.work_life_span],color="b",alpha=0.15, label='$L_a$')
    unemployed = ax.fill_between(range(0,par.work_life_span),ss.L_a[0:par.work_life_span],1,color="r",alpha=0.15, label='$U_a$')
    m_s = ax.hlines(ss.m_s, 0, par.work_life_span, colors='orange',linestyles='dashed', label='$m^s$')
    ax.legend(handles = [employed,unemployed,m_s], loc=1)
    ax.set_title('Employed, $L_a$, and unemployed, $U_a$');

In [ ]:
widgets.interact(household_search_figure,
    delta_L_a_fac=widgets.FloatSlider(description="delta_L_a_fac", min=0.0, max=1, value=par.delta_L_a_fac));

**Reset demographic structure:**

In [ ]:
model.demographic_structure(par.delta_L_a_fac)      #reset the demographic structure
household_search_ss(par,ss)                         #find household search using default demographic structure 

## 4. <a id='toc4_'></a>[Consumption-saving](#toc0_)

**Required inputs:**

In [7]:
ss.W = par.W_ss                                 #Set steady state wage
ss.pi_hh = par.pi_hh_ss                         #Set steady state inflation
ss.P_C = 1.0                                    #Set steady state consumer price
ss.tau = 0.40                                   #Set steady state tax rate
ss.Aq = Aq_ini = 0.0                            #Set steady state bequests
ss.r_hh = par.r_hh                              #Set steady state interest rate
ss.real_r_hh = (1+ss.r_hh)/(1+ss.pi_hh)-1       #Set steady state real interest rate

**Solve:**

In [8]:
find_household_consumption_ss(model)            #find household consumption behavior
find_Aq_ss(ss.Aq,model)                         #find household bequests behavior

In [ ]:
def plot_CA():
    
    fig = plt.figure(figsize=(3*6,1*6/1.5))

    ax = fig.add_subplot(1,3,1)
    ax.plot(ss.inc_a,label='all',color='black')
    ax.set_title('Income');
    ax.set_ylim([0,1.2])
    
    ax = fig.add_subplot(1,3,2)
    ax.plot(ss.C_R_a,ls='--',label='Ricardian')
    ax.plot(ss.C_HtM_a,ls='--',label='HtM')
    ax.plot(ss.C_a,label='all',color='black')
    ax.set_title('Consumption, $C_a$');
    ax.set_ylim([0,1.2])
    
    ax = fig.add_subplot(1,3,3)
    ax.plot(ss.A_R_a,ls='--',label='Ricardian')
    ax.plot(ss.A_HtM_a,ls='--',label='HtM')
    ax.plot(ss.A_a,label='all',color='black')
    ax.set_title('Savings, $A_a$')
    ax.set_ylim([-2,10])
    ax.legend(frameon=True)
    
plot_CA()    

## 5. <a id='toc5_'></a>[Varying central inputs](#toc0_)

In [ ]:
def compare_models(model,name,value,ns='ss'):

    model_ = model.copy()
    
    if ns == 'ss':
        model_.ss.__dict__[name] = value
        oldvalue = model.ss.__dict__[name]
    elif ns == 'par':
        model_.par.__dict__[name] = value
        oldvalue = model.par.__dict__[name]
    else:
        raise NotImplemented
            
    model_.ss.real_r_hh = (1+model_.par.r_hh)/(1+model_.ss.pi_hh)-1            
    find_household_consumption_ss(model_)

    fig = plt.figure(figsize=(3*6,1*6/1.5))


    ax = fig.add_subplot(1,3,1)
    ax.plot(ss.inc_a,ls='--',label=f'{name} = {oldvalue:.2f}')
    ax.plot(model_.ss.inc_a,ls='--',label=f'{name} = {value:.2f}')
    ax.set_title('Income');
    ax.set_ylim([0,1.2])
    ax.legend(frameon=True)
    
    ax = fig.add_subplot(1,3,2)
    ax.plot(ss.C_a,ls='--',label=f'{name} = {oldvalue:.2f}')
    ax.plot(model_.ss.C_a,ls='--',label=f'{name} = {value:.2f}')
    ax.set_title('Consumption, $C_a$');
    ax.set_ylim([0,1.2])
    
    ax = fig.add_subplot(1,3,3)
    ax.plot(ss.A_a,ls='--',label=f'{name} = {oldvalue:.2f}')
    ax.plot(model_.ss.A_a,ls='--',label=f'{name} = {value:.2f}')
    ax.set_title('Savings, $A_a$')
    ax.set_ylim([-2,10])
    
compare_models(model,'Aq',ss.Aq*0.0)
compare_models(model,'mu_Aq',par.mu_Aq/2,ns='par')
compare_models(model,'W',ss.W*1.1)
compare_models(model,'W_U',par.W_U*1.1,ns='par')
compare_models(model,'W_R',par.W_R*1.1,ns='par')
compare_models(model,'tau',ss.tau*0.9)
compare_models(model,'beta',par.beta+0.02,ns='par')
compare_models(model,'r_hh',par.r_hh-0.02,ns='par')
compare_models(model,'Lambda',par.Lambda/2,ns='par')

## 6. <a id='toc6_'></a>[Test household blocks](#toc0_)

In [9]:
sol.A_R_death[:] = ss.A_R_a[-1]                     #Set solution for last period assets
sol.Aq[:] = ss.Aq                                   #Set solution for bequests
sol.P_C[:] = ss.P_C                                 #Set solution for consumer price
sol.W[:] = ss.W                                     #Set solution for wage
sol.tau[:] = ss.tau                                 #Set solution for tax rate
sol.L[:] = ss.L                                     #Set solution for labor supply
sol.m_s[:] = ss.m_s                                 #Set solution for job-finding rate
sol.r_hh[:] = ss.r_hh                               #Set solution for interest rate
sol.real_r_hh[:] = ss.real_r_hh                     #Set solution for real interest rate

In [11]:
model.ini = ss                                      #Set model initial to steady state

In [12]:
model.evaluate_block('search_and_match')            #Evaluate the search and matching behavior

In [ ]:
assert np.allclose(sol.S_a-ss.S_a[:,np.newaxis],0.0)
assert np.allclose(sol.L_ubar_a-ss.L_ubar_a[:,np.newaxis],0.0)
assert np.allclose(sol.L_a-ss.L_a[:,np.newaxis],0.0)
assert np.allclose(sol.U_a-ss.U_a[:,np.newaxis],0.0)
assert np.allclose(sol.S-ss.S,0.0)
assert np.allclose(sol.L_ubar-ss.L_ubar,0.0)
assert np.allclose(sol.U-ss.U,0.0)

In [13]:
model.evaluate_block('household_consumption')       #Evaluate the household consumption behavior

In [ ]:
assert np.allclose(sol.Aq_diff,0.0)
assert np.allclose(sol.A_R_ini_error,0.0)